In [ ]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sn

#Scalers
from sklearn.preprocessing import StandardScaler

In [ ]:
# loading the data
df1 = pd.read_csv('train.csv')

In [ ]:
df1.head()

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
3,A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
4,A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan


In [ ]:
df1.shape

(26729, 10)

In [ ]:
# drop outcomesubtype - not relevant for model
df1 = df1.drop(['OutcomeSubtype'], axis=1)

In [ ]:
# missing data

total = df1.isnull().sum().sort_values(ascending=False)
percent = (df1.isnull().sum()/df1.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent * 100], axis=1, keys=['Total', 'Percent(%)'])
missing_data.head(10)

,Total,Percent(%)
Name,7691,28.773991
AgeuponOutcome,18,0.067343
SexuponOutcome,1,0.003741
AnimalID,0,0.000000
DateTime,0,0.000000
OutcomeType,0,0.000000
AnimalType,0,0.000000
Breed,0,0.000000
Color,0,0.000000


In [ ]:
df1 = df1.loc[df1['AgeuponOutcome'].notnull()]
df1 = df1.loc[df1['SexuponOutcome'].notnull()]

In [ ]:
total = df1.isnull().sum().sort_values(ascending=False)
percent = (df1.isnull().sum()/df1.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent * 100], axis=1, keys=['Total', 'Percent(%)'])
missing_data.head(10)

,Total,Percent(%)
Name,7673,28.727069
AnimalID,0,0.000000
DateTime,0,0.000000
OutcomeType,0,0.000000
AnimalType,0,0.000000
SexuponOutcome,0,0.000000
AgeuponOutcome,0,0.000000
Breed,0,0.000000
Color,0,0.000000


In [ ]:
df1.shape

(26710, 9)